#### Just for experimentation, use train_lstm.py!!!

In [1]:
# This Python file uses the following encoding: utf-8
from keras.models import Sequential
from keras.models import model_from_json
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

from lib.pretty_testing import predict_test

import numpy as np
import pickle

Using TensorFlow backend.


In [4]:
dataset_file = 'embedded_docs_with_verb.p'
model_file = 'models/keras_model_2.json'
model_weights_file = 'models/keras_new_weights_with_verb.h5'

In [3]:
# fix random seed for reproducibility
np.random.seed(8)


# load prepared data
with open(dataset_file) as f:
    data, labels = pickle.load(f)

In [4]:
# padding for the rnn
padded_data = sequence.pad_sequences(data, maxlen=200,padding="pre", truncating="post", value=0.0, dtype='float32')

In [5]:
# free ram from the original dataset
del data

In [6]:
# split dataset
X_train, X_test, y_train, y_test = train_test_split(padded_data, labels, train_size=0.9, stratify=labels)
del padded_data

/usr/local/lib/python2.7/dist-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [5]:
model = Sequential()
model.add(LSTM(100, input_shape = (200, 100), return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

'''
model = Sequential()
model.add(LSTM(100, input_shape = (200, 100)))
#model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=lr), metrics=['accuracy'])
print(model.summary())
'''


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 200, 100)          80400     
_________________________________________________________________
lstm_5 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 110,651
Trainable params: 110,651
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
with open(model_file,'w') as f:
    f.write(model.to_json())

In [ ]:
model.fit(X_train, y_train, epochs=3, batch_size=64, )
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# serialize weights to HDF5
model.save_weights(model_weights_file)

### Test loading

In [ ]:
model.load_weights("models/keras_weights.h5")

In [ ]:
predict_test(model, X_test, y_test, ['non_cost', 'cost'])

In [ ]:
with open('models/keras_model.json') as f:
    model = model_from_json(f.read())
model.summary()